# SUDO CODE 2025

- Name: Nguyễn Anh Đức
- Team: 9
- ID: 28

## Import libs

In [46]:
import json
import zipfile
import pandas as pd
import numpy as np
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize

## Exploring data

### Read rawdata from file

In [2]:
zip_file_path = 'data/news_dataset.json.zip'
extraction_path = 'data'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [3]:
raw_df = pd.read_json("data/news_dataset.json")

In [4]:
raw_df.head()

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,218266,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170


### Number of rows and columns

In [5]:
shape = raw_df.shape
print(shape)

(184539, 10)


### Check duplicate rows

In [6]:
num_duplicated_rows = raw_df.duplicated().sum()
print(num_duplicated_rows)

0


### Remove empty data

In [7]:
raw_df = raw_df.dropna() 
raw_df = raw_df[~raw_df.eq('').any(axis=1)]
raw_df = raw_df.reset_index(drop=True)

In [8]:
print(raw_df.shape)

(113418, 10)


### Meaning of each column

- `id`: Primary key in the database, used only for identification.  
- `author`: Name of the person or group of journalists who wrote the article.  
- `title`: Title of the article.  
- `content`: Text content of the article.  
- `picture_count`: Number of pictures used in the article.  
- `topic`: Main topic of the article.  
- `url`: URL of the article.  
- `source`: News outlet that published the article.  
- `crawled_at`: The time when the article was crawled (usually very close to the upload time).  
- `processed`: Can be ignored.  

In [9]:
raw_df = raw_df.drop(columns=['processed'])

### Columns' data type

In [10]:
dtypes = raw_df.dtypes
print(dtypes)

id                        int64
author                   object
content                  object
picture_count             int64
source                   object
title                    object
topic                    object
url                      object
crawled_at       datetime64[ns]
dtype: object


### Exploring numerical attributes

In [11]:
num_col_df = raw_df.select_dtypes(exclude=['object', 'datetime64','datetime64[ns, UTC]'])
num_col_df.head()

,id,picture_count
0,218269,1
1,218268,3
2,218267,1
3,218266,12
4,218261,1


- Since the `id` column is the primary key in the database, representing the identity of each news item, we will exclude it from the following exploratory steps for numerical attributes.

In [12]:
num_col_df = num_col_df.drop(columns=["id"])

In [13]:
def median(df):
    return df.median()

def lower_quartile(df):
    return df.quantile(0.25)

def upper_quartile(df):
    return df.quantile(0.75)

In [14]:
num_col_info_df = num_col_df.agg(["min", lower_quartile, median, upper_quartile, "max"])
num_col_info_df

,picture_count
min,0.0
lower_quartile,1.0
median,1.0
upper_quartile,2.0
max,74.0


### Exploring non-numerical attributes

In [15]:
cate_col_df = raw_df.select_dtypes(include=['object'])
cate_col_df.head()

,author,content,source,title,topic,url
0,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...
1,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...
2,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...
3,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...
4,Đỗ Trung,Siêu sao Bồ Đào Nha được HLV Erik ten Hag bố t...,bongdaplus,"Bị thay sau hiệp 1, Ronaldo đùng đùng bỏ về sớm",Ngoại hạng Anh,https://bongdaplus.vn/ngoai-hang-anh/ronaldo-d...


In [16]:
def num_values(s):
    return s.nunique()

def value_ratios(s):
    return (s.dropna().value_counts(normalize=True) * 100).round(1).to_dict()

In [17]:
cat_col_info_df = cate_col_df.agg([num_values, value_ratios])
cat_col_info_df

,author,content,source,title,topic,url
num_values,17282,110437,24,106972,1347,113418
value_ratios,"{'PV/VOV.VN': 0.6, 'Trường Thịnh': 0.6, 'Ánh D...",{'BÁO ĐIỆN TỬ THỂ THAO & VĂN HÓA - TTXVN Giấy ...,"{'thanhnien.vn': 10.1, 'vnexpress': 7.5, 'dant...",{'Không tự ý mua thuốc Tamiflu điều trị cúm': ...,"{'Thế giới': 7.6, 'Thể thao': 6.8, 'Xã hội': 5...",{'https://vtc.vn/bo-qua-mang-5g-nga-tien-thang...


## Preprocessing data

In [18]:
raw_df['content_orig'] = raw_df['content'].astype(str)

### Lowercase the content

In [20]:
raw_df["content"] = raw_df["content"].str.lower()
raw_df['content'].head()

0    gần đây, thứ trưởng bộ phát triển kỹ thuật số,...
1    kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...
2    thống đốc kentucky andy beshear hôm 31/7 cho h...
3    vụ tai nạn giao thông liên hoàn trên phố đi bộ...
4    siêu sao bồ đào nha được hlv erik ten hag bố t...
Name: content, dtype: object

### Remove HTML tags

In [ ]:
raw_df['content'] = raw_df['content'].apply(lambda t: re.sub(r'<[^>]*>', ' ', t))

### Remove URLs

In [22]:
raw_df['content'] = raw_df['content'].apply(lambda t: re.sub(r'https?:\/\/\S+|www\.\S+', ' ', t))

### Remove punctuation

In [23]:
raw_df['content'] = raw_df['content'].apply(lambda t: re.sub(r'[^\w\s]', ' ', t, flags=re.UNICODE))

### Collapse multiple whitespaces and strip

In [24]:
raw_df['content'] = raw_df['content'].apply(lambda t: re.sub(r'\s+', ' ', t).strip())

### Drop rows that became empty after cleaning and reindex

In [25]:
raw_df = raw_df[raw_df['content'].str.strip() != ''].reset_index(drop=True)
raw_df.shape

(113298, 10)

## Tokenization

In [34]:
raw_df['tokens'] = raw_df['content'].apply(lambda x: word_tokenize(x))
raw_df[['content', 'tokens']].head()

,content,tokens
0,gần đây thứ trưởng bộ phát triển kỹ thuật số t...,"[gần, đây, thứ, trưởng, bộ, phát, triển, kỹ, t..."
1,kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...,"[kết, quả, thi, tốt, nghiệp, thpt, năm, 2022, ..."
2,thống đốc kentucky andy beshear hôm 31 7 cho h...,"[thống, đốc, kentucky, andy, beshear, hôm, 31,..."
3,vụ tai nạn giao thông liên hoàn trên phố đi bộ...,"[vụ, tai, nạn, giao, thông, liên, hoàn, trên, ..."
4,siêu sao bồ đào nha được hlv erik ten hag bố t...,"[siêu, sao, bồ, đào, nha, được, hlv, erik, ten..."


## Stopword removal

In [40]:
all_words = [word for tokens in raw_df['tokens'] for word in tokens]
word_count = Counter(all_words)
stop_words = [word for word, _ in word_count.most_common(50)]
print("Chosen stopwords:", stop_words)

Chosen stopwords: ['và', 'của', 'có', 'các', 'là', 'trong', 'với', 'cho', 'được', 'không', 'đã', 'người', 'một', 'công', 'để', 'khi', 'những', 'ở', 'này', 'thể', 'đến', 'từ', 'đó', 'về', 'năm', 'tại', 'đồng', 'nhiều', 'cũng', 'ra', 'vào', 'theo', 'việc', 'hiện', 'sẽ', 'đầu', 'trên', 'sự', 'thành', 'số', 'động', 'ngày', 'nhà', 'bị', 'như', 'nam', 'định', 'làm', 'quan', 'gia']


In [41]:
def remove_stopwords(tokens, stop_words):
    return [word for word in tokens if word not in stop_words]

raw_df['tokens_no_stop'] = raw_df['tokens'].apply(lambda x: remove_stopwords(x, stop_words))
raw_df[['tokens', 'tokens_no_stop']].head()

,tokens,tokens_no_stop
0,"[gần, đây, thứ, trưởng, bộ, phát, triển, kỹ, t...","[gần, đây, thứ, trưởng, bộ, phát, triển, kỹ, t..."
1,"[kết, quả, thi, tốt, nghiệp, thpt, năm, 2022, ...","[kết, quả, thi, tốt, nghiệp, thpt, 2022, thấy,..."
2,"[thống, đốc, kentucky, andy, beshear, hôm, 31,...","[thống, đốc, kentucky, andy, beshear, hôm, 31,..."
3,"[vụ, tai, nạn, giao, thông, liên, hoàn, trên, ...","[vụ, tai, nạn, giao, thông, liên, hoàn, phố, đ..."
4,"[siêu, sao, bồ, đào, nha, được, hlv, erik, ten...","[siêu, sao, bồ, đào, nha, hlv, erik, ten, hag,..."
